# Mission to Mars

In [1]:
# Dependencies
import os
import html5lib
import numpy as np
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import requests
import re
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url= 'https://mars.nasa.gov/news/'

In [5]:
browser.visit(url)
time.sleep(3)
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
slides = soup.find_all('li', class_='slide')

In [7]:
content_title = slides[0].find('div', class_ = 'content_title')
news_title = content_title.text.strip()
article_teaser_body = slides[0].find('div', class_ = 'article_teaser_body')
news_p = article_teaser_body.text.strip()
print("Title: ",news_title)
print("Paragraph: ",news_p)

Title:  Independent Review Indicates NASA Prepared for Mars Sample Return Campaign
Paragraph:  NASA released an independent review report Tuesday indicating the agency is well positioned for its Mars Sample Return campaign to bring pristine samples from Mars to Earth for scientific study.


# JPL Mars Space Images

In [8]:
# url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
base_url = 'https://www.jpl.nasa.gov'
url = base_url + '/spaceimages/?search=&category=Mars'

In [9]:
browser.visit(url)
time.sleep(3)
html = browser.html
soup = bs(html, 'html.parser')

In [10]:
image_url=soup.find("a", class_ = "button fancybox")["data-fancybox-href"]
featured_image_url = base_url + image_url
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA19685_ip.jpg


# Mars Hemishperes

In [12]:
base_url = 'https://astrogeology.usgs.gov'
url = base_url + '/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [13]:
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [14]:
items = soup.find_all('div', class_='item')

In [15]:
urls = []
titles = []
for item in items:
    urls.append(base_url + item.find('a')['href'])
    titles.append(item.find('h3').text.strip())
print(urls)
titles

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced', 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']


['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [16]:
browser.visit(urls[0])
html = browser.html
soup = bs(html, 'html.parser')
oneurl = base_url+soup.find('img',class_='wide-image')['src']
oneurl

'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'

In [17]:
img_urls = []
for oneurl in urls:
    browser.visit(oneurl)
    html = browser.html
    soup = bs(html, 'html.parser')
#     savetofile(textfilename,soup.prettify())
    oneurl = base_url+soup.find('img',class_='wide-image')['src']
    img_urls.append(oneurl)
    
img_urls

['https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [18]:
hemisphere_image_urls = []

for i in range(len(titles)):
    hemisphere_image_urls.append({'title':titles[i],'img_url':img_urls[i]})

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [19]:
for i in range(len(hemisphere_image_urls)):
    print(hemisphere_image_urls[i]['title'])
    print(hemisphere_image_urls[i]['img_url'] + '\n')

Cerberus Hemisphere Enhanced
https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg

Schiaparelli Hemisphere Enhanced
https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg

Syrtis Major Hemisphere Enhanced
https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg

Valles Marineris Hemisphere Enhanced
https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg



# Mars Facts

In [20]:
facts_url = 'https://space-facts.com/mars/'
tables = pd.read_html(facts_url)
tables

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:852)>

In [28]:
print(f'Total tables: {len(tables)}')

NameError: name 'tables' is not defined